In [30]:
from langchain.chat_models import ChatOpenAI
from typing import Text
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

cache_dir = LocalFileStore("./.cache/")

llm = ChatOpenAI(temperature=0.1)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="Commentaires",
    chunk_size=600,
    chunk_overlap=100
)


loader = UnstructuredFileLoader("./files/plan_weekly.xlsx")
docs = loader.load_and_split(text_splitter=splitter)


Created a chunk of size 861, which is longer than the specified 600
Created a chunk of size 1573, which is longer than the specified 600
Created a chunk of size 1637, which is longer than the specified 600
Created a chunk of size 1581, which is longer than the specified 600
Created a chunk of size 1539, which is longer than the specified 600
Created a chunk of size 1801, which is longer than the specified 600
Created a chunk of size 1686, which is longer than the specified 600


In [31]:
for doc in docs:
    doc.page_content = doc.page_content.replace('\n', '')


In [32]:
embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

# prompt = ChatPromptTemplate.from_messages([
    # ("system",
    #  "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer, just say you don't know. Don't make it up:\n{context}"),
    # ("human", "{question}")
# ])

# chain = {"context": retriever, "question": RunnablePassthrough(
# )} | prompt | llm


In [33]:
map_doc_prompt = ChatPromptTemplate.format_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim.
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)


map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
    documents = inputs['documents']
    question = inputs['question']
    return "\n\n".join(
        map_doc_chain.invoke({"context":doc.page_content, "question":question}).content
        for doc in documents
    )
    


map_chain = {"documents": retriever, "question": RunnablePassthrough()} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_message([
    ("system",
     "Given the following extracted parts of a long document and a question, create a final answer. If you don't know the answer, just say you don't know. Don't make it up:\n{context}"),
    ("human", "{question}")
])


chain = {"context": map_chain, "question":RunnablePassthrough()} |final_prompt | llm


AIMessage(content='목요일에는 출장으로 대전에 갔다가 회의를 진행하고 돌아온 후에 메일 처리와 아이 돌봄을 하였습니다. 또한, 지하 매점에서 와인을 구매하였습니다.')